In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os

C:\Users\Saranga\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Saranga\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Saranga\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Saranga\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:

from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Dense, Activation,Flatten, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential,Model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [4]:
img_size=[227,227]

In [5]:
num_classes=4

In [6]:
class_map={
    "rock":0,
    "paper":1,
    "scissors":2,
    "none":3
}

In [7]:
def mapper(val):
    return class_map[val]

In [8]:
train_path='image_data'

In [9]:
vgg=VGG16(input_shape=img_size+[3],weights='imagenet',include_top=False)

In [10]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
for layer in vgg.layers:
    layer.trainable=False

In [12]:
x=Dropout(0.4)(vgg.output)
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
pred=Dense(4,activation='softmax')(x)


In [13]:
model=Model(inputs=vgg.input,outputs=pred)

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [16]:
train_data=ImageDataGenerator(rescale=1./255,shear_range=0.2)

In [17]:
train_set=train_data.flow_from_directory(train_path,target_size=(227,227),class_mode='categorical',shuffle=True)

Found 1116 images belonging to 4 classes.


In [18]:
r=model.fit_generator(train_set,epochs=10,verbose=1)


Epoch 1/10
35/35 [==============================] - 130s 4s/step - loss: 0.6804 - accuracy: 0.8145
Epoch 2/10
35/35 [==============================] - 76s 2s/step - loss: 0.0279 - accuracy: 0.9919
Epoch 3/10
35/35 [==============================] - 83s 2s/step - loss: 0.0089 - accuracy: 0.9973
Epoch 4/10
35/35 [==============================] - 73s 2s/step - loss: 0.0044 - accuracy: 1.0000
Epoch 5/10
35/35 [==============================] - 73s 2s/step - loss: 0.0030 - accuracy: 1.0000
Epoch 6/10
35/35 [==============================] - 75s 2s/step - loss: 0.0031 - accuracy: 1.0000
Epoch 7/10
35/35 [==============================] - 75s 2s/step - loss: 0.0028 - accuracy: 1.0000
Epoch 8/10
35/35 [==============================] - 76s 2s/step - loss: 0.0020 - accuracy: 1.0000
Epoch 9/10
35/35 [==============================] - 75s 2s/step - loss: 0.0016 - accuracy: 1.0000
Epoch 10/10
35/35 [==============================] - 76s 2s/step - loss: 0.0012 - accuracy: 1.0000


In [19]:
model.save("rps_vgg16.h5")

In [15]:
from keras.models import load_model
import cv2
import numpy as np
import sys

filepath=r'‪C:\Users\Saranga\Desktop\DSC_0339.JPG'

model=load_model("rps_vgg16.h5")

img=cv2.imread(filepath)
print(img)

img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=cv2.resize(img,(227,227))

pred=model.predict(np.array([img]))

print(pred)


None


In [17]:
from keras.models import load_model
import cv2
import numpy as np
from random import choice

REV_CLASS_MAP = {
    0: "rock",
    1: "paper",
    2: "scissors",
    3: "none"
}


def mapper(val):
    return REV_CLASS_MAP[val]


def calculate_winner(move1, move2):
    if move1 == move2:
        return "Tie"

    if move1 == "rock":
        if move2 == "scissors":
            return "User"
        if move2 == "paper":
            return "Computer"

    if move1 == "paper":
        if move2 == "rock":
            return "User"
        if move2 == "scissors":
            return "Computer"

    if move1 == "scissors":
        if move2 == "paper":
            return "User"
        if move2 == "rock":
            return "Computer"


model = load_model("rps_vgg16.h5")

cap = cv2.VideoCapture(0)

prev_move = None

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # rectangle for user to play
    cv2.rectangle(frame, (100, 100), (500, 500), (255, 255, 255), 2)
    # rectangle for computer to play
    cv2.rectangle(frame, (800, 100), (1200, 500), (255, 255, 255), 2)

    # extract the region of image within the user rectangle
    roi = frame[100:500, 100:500]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (227, 227))

    # predict the move made
    pred = model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "none":
            computer_move_name = choice(['rock', 'paper', 'scissors'])
            winner = calculate_winner(user_move_name, computer_move_name)
        else:
            computer_move_name = "none"
            winner = "Waiting..."
    prev_move = user_move_name

    # display the information
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (50, 50), font, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (750, 50), font, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Winner: " + winner,
                (400, 600), font, 2, (0, 0, 255), 4, cv2.LINE_AA)

    if computer_move_name != "none":
        icon = cv2.imread(
            "images/{}.png".format(computer_move_name))
        icon = cv2.resize(icon, (400, 400))
        frame[100:500, 800:1200] = icon

    cv2.imshow("Rock Paper Scissors", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[1,64,227,227] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv1_11/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[1,64,227,227] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv1_11/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[dense_2_11/Softmax/_3575]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors igno